In [204]:
import numpy as np
import matplotlib.pyplot as plt
import sys

# Constants
I = np.eye(3) # Identity matrix
R = 6371000   # Earth radius               [m]
g = 9.81      # Gravitational acceleration [g/ms^2]
U = 7.27e-5   # Earth angular rotation     [rad/s]     
t = 5e3       # Simulation time            [sec]
mu    = 0     # Expected value
sigma = 1     # SKO
b = 5e-8      # Сorrelation parameter (beta)
A = 1e-4      # Constant ??
rate = 1      # Model rate                 [Hz]

iter = int(t * 1/rate) - 1 # Model iterrations count [n]

# Set initial conditions (all with 'dots')
dVe_0  = 0
Fn_0   = 0
Wdrn_0 = 2e-7

In [205]:
# Programm control flags
generate_new_clear_params = False # Флаг сохранения получившихся массивов в файл
generate_new_noize_params = False # Флаг сохранения получившихся зашумленных массивов в файл

plot_clear_graphs_flag     = False
plot_noize_graphs_flag     = False
plot_mesuarment_noize_flag = False

In [206]:
# Plot function
def plot(data, xlabel, ylabel, title, legend=None):
    width = 20
    height = 12
    
    plt.figure(figsize=(width, height))    
    
    for each in data:
        plt.plot(each)
    plt.suptitle(title, fontsize=20)
    plt.xlabel(xlabel, fontsize = 30)
    plt.ylabel(ylabel, fontsize = 30)
    plt.xticks(fontsize = 20)
    plt.yticks(fontsize = 20)
    if legend != None:
        plt.legend(legend, prop={'size': 20})
    plt.grid()
    plt.show()

In [207]:
#### Clear graphs ================================================================================== 
    
# Чистые, незашумленные графики, их нужно сгенерировать 1 раз и сохранить
# в файл
    
# Если флаг 'generate_new_clear_params' установлен (= True), то генерируется новая траектория, 
# которая записывается в массивы и в файл clear_params.txt. Иначе, парсим файл clear_params.txt,
# записываем данные отуда в массивы.

dVe_clear  = []  # Clear linear velocity
Fn_clear   = []   # Clear angle
Wdrn_clear = [] # Clear angular velocity

def form_clear_parameters(dVe_clear_, Fn_clear_, Wdrn_clear_):
    if generate_new_clear_params:
        # Generating new model parmas by rewriting 'clear_params.txt' file
        print('Generating new model params ...')
        f = open('./clear_params.txt', 'w')
        
        dVe_clear.appned(dVe_0)
        Fn_clear.append(Fn_0)
        Wdrn_clear.append(Wdrn_0)
        
        # Fill file witch initial conditions
        f.write("{} {} {}\n".format(dVe_0, Fn_0, Wdrn_0))
        for i in range(1, iter):
            dVe_clear_.append(dVe_clear[i - 1] - g * Fn_clear[i - 1])
            Fn_clear_.append(Fn_clear[i - 1] + dVe_clear[i - 1] / R + Wdrn_clear[i - 1])
            Wdrn_clear_.append(Wdrn_clear[i - 1])

            f.write("{} {} {}\n".format(dVe_clear[i], Fn_clear[i], Wdrn_clear[i]))
        print('Model params generated successfully')
    else:
        print('Setting model params from file ...')
        f = open('./clear_params.txt', 'r')
        params = f.read()
        if len(params) == 0:
            print("Unable to read params from file: File is empty")
            print("set 'generate_new_params' to 'True', to fill file")
            sys.exit(-1)
        
        lines = params.split('\n')
        for line in lines:
            if line != '':
                ve, fn, wdr = line.split()
                dVe_clear_.append(float(ve))
                Fn_clear_.append(float(fn))
                Wdrn_clear_.append(float(wdr))

        print('Model params setted successfully')
    
    f.close()
        
def plot_clear_gparhs():
    plot([dVe_clear], "Time, s", "Ve, m/s", 'Clear')
    plot([Fn_clear], "Time, S", "Fn, rad", 'Clear')
    plot([Wdrn_clear], "Time, S", "Wdrn, rad/s", 'Clear')

form_clear_parameters(dVe_clear, Fn_clear, Wdrn_clear)
if plot_clear_graphs_flag:
    plot_clear_gparhs()

Setting model params from file ...
Model params setted successfully


In [208]:
# Matrix graphs ================================================================================= 

# Geerating clear model parametrs using matrices
# 

A1 = np.array([[0,  -g, 0     ], 
                [1/R, 0, 1/rate],
                [0,   0, -b   ]])

F = I + A1

x = [np.array([[0], 
               [0],
               [Wdrn_0]])]

# Setting initial condition
dVe_matrix  = [dVe_0]
Fn_matrix   = [Fn_0]
Wdrn_matrix = [Wdrn_0]

def form_matrix_graphs():
    for i in range(1, iter):
        x.append(np.dot(F , x[i - 1]))
        dVe_matrix.append(x[i][0])
        Fn_matrix.append(x[i][1])
        Wdrn_matrix.append(x[i][2])
        
def plot_matrix_graphs():
    
    plot(dVe_matrix, "Time, s", "Ve, m/s", 'Matrix')
    plot(Fn_matrix, "Time, S", "Fn, rad", 'Matrix')
    plot(Wdrn_matrix, "Time, S", "Wdrn, rad/s", 'Matrix')
    
form_matrix_graphs()
# plot_matrix_graphs()

In [209]:
# Noize graphs ==================================================================================
W = np.random.normal(mu, sigma, iter)

dVe_noize_0  = 0
Fn_noize_0   = 0
Wdrn_noize_0 = Wdrn_0

F = np.array([[1,  -g/rate, 0],
              [1/R/rate, 1, 1],
              [0,   0, (1 - b) * 1/rate]])

X = [np.array([[dVe_noize_0], 
               [Fn_noize_0],
               [Wdrn_noize_0]])]

dVe_noize  = []
Fn_noize   = []
Wdrn_noize = []

G = np.array([[0.], 
              [0.],
              [A * np.sqrt(2 * b)]])

def form_noize_graphs(dVe_noize, Fn_noize, Wdrn_noize):
    if generate_new_noize_params:
        print('Generating new noized params ...')
        f = open('./noize_params.txt', 'w')
        
        dVe_noize.appned(dVe_noize_0)
        Fn_noize.append(Fn_noize)
        Wdrn_noize.append(Wdrn_noize)
        
        f.write("{} {} {}\n".format(dVe_noize[0], Fn_noize[0], Wdrn_noize[0]))
        for i in range(1, iter):
            X.append(np.dot(F , X[i - 1]) + G * W[i])
            dVe_noize.append(X[i][0][0])
            Fn_noize.append(X[i][1][0])
            Wdrn_noize.append(X[i][2][0])
            
            f.write("{} {} {}\n".format(dVe_noize[i], Fn_noize[i], Wdrn_noize[i]))
            
        print('Noized params generated successfully')
    else:
        print('Setting noized params from file ...')
        f = open('./noize_params.txt', 'r')
        params = f.read()
        if len(params) == 0:
            print("Unable to read params from file: File is empty")
            print("set 'generate_new_params' to 'True', to fill file")
            sys.exit(-1)
        
        
        lines = params.split('\n')
        for line in lines:
            if line != '':
                ve, fn, wdr = line.split()
                dVe_noize.append(float(ve))
                Fn_noize.append(float(fn))
                Wdrn_noize.append(float(wdr))
            
        print('Noized params setted successfully')
        
    
def plot_noize_graphs():
    plot([dVe_noize], "Time, s", "Ve, m/s", 'Noize')
    plot([Fn_noize], "Time, S", "Fn, rad", 'Noize')
    plot([Wdrn_noize], "Time, S", "Wdrn, rad/s", 'Noize')

form_noize_graphs(dVe_noize, Fn_noize, Wdrn_noize)
if plot_noize_graphs_flag: 
    plot_noize_graphs()

Setting noized params from file ...
Noized params setted successfully


In [210]:
dVe_mesuarment_noize  = []
Fn_mesuarment_noize   = []
Wdrn_mesuarment_noize = []
W_mesuarment_noize = np.random.normal(mu, sigma, iter)

def form_mesuarment_noize(dVe_mesuarment_noize, Fn_mesuarment_noize, Wdrn_mesuarment_noize, W_mesuarment_noize):
    for i in range(0, iter-1):
        dVe_mesuarment_noize.append(dVe_noize[i]  + W_mesuarment_noize[i] * 0.1)
        Fn_mesuarment_noize.append(Fn_noize[i]   + W_mesuarment_noize[i] * 1e-5)
        Wdrn_mesuarment_noize.append(Wdrn_noize[i] + W_mesuarment_noize[i] * 1e-3)
        
def plot_mesuarment_noize():
    plot([dVe_mesuarment_noize], "Time, s", "Ve, m/s", 'dVe Measure noize')
    plot([Fn_mesuarment_noize], "Time, s", "Fn, rad", 'Fn Measure noize')
    plot([Wdrn_mesuarment_noize], "Time, s", "Wdrn, rad/s", 'Wdrn Measure noize')

form_mesuarment_noize(dVe_mesuarment_noize, Fn_mesuarment_noize, Wdrn_mesuarment_noize, W_mesuarment_noize)
if plot_mesuarment_noize_flag:
    plot_mesuarment_noize()

In [211]:
r = 1e-2
Q = 1e-13

H = np.array([[1, 0, 0]])

F = np.array([[1,  -g, 0], 
              [rate/R, 1, 1],
              [0,   0, 1 - b]])

X_filtred = [np.array([[0],
                       [0],
                       [0]])]

P_cur = [np.array([[1e3, 0, 0],
                [0, 1e-3, 0],
                [0, 0, 1e-12]])]

X_prev_h = [np.array([[0],
                      [0],
                      [0]])]



P = [np.array([[0, 0, 0],
                [0, 0, 0],
                [0, 0, 0]])]


G = np.array([[0.],
              [0.],
              [1]])

K = [0]

GQT = np.array([[0, 0, 0],
                 [0, 0, 0],
                 [0, 0, Q]])




IndexError: list index out of range

In [ ]:
def filter():
    r = 1e-2
    Q = 1e-15
    
    H = np.array([[1, 0, 0]])

    F = np.array([[1,     -g * rate, 0], 
                  [rate/R, 1,        1],
                  [0,      0,    1 - b]])
    
    # Начальные условия
    ev_cur = [np.array([[0], 
                    [0],
                    [0]])]



    P_cur = [np.array([[0, 0, 0], 
                   [0, 0, 0],
                   [0, 0, 0]])]


    K = [0]
              
    ev_x = [np.array([[0], 
                  [0],
                  [0]])]
          
    
    P = [np.array([[0, 0, 0], 
               [0, 0, 0],
               [0, 0, 0]])]


    G = np.array([[0.], 
              [0.],
              [1]])               

    for i  in range(1,iter):
    
    
        ev_cur.append(np.dot(F, ev_x[i - 1]))
    
    
        P_cur.append(np.dot(np.dot(F, P[i - 1]), np.transpose(F)) + (np.dot(np.dot(G, Q), np.transpose(G))))
    
    
        K_num = np.dot(P_cur[i], np.transpose(H))
        K_den = np.dot(np.dot(H, P_cur[i]), np.transpose(H)) + r
        K_den_inv = np.linalg.inv(K_den)
        K.append(np.dot(K_num, K_den_inv))

        
        ev_x.append(ev_cur[i] + np.dot(K[i], (z[i - 1] - np.dot(H , ev_cur[i]))))
    
    
        P.append(np.dot((np.eye(3) - np.dot(K[i], H)), P_cur[i]))
    

    v = []
    f = []
    w = []

    for i in range(iter-1):
        v.append(ev_cur[i]#!/usr/bin/python3.8[0])
        f.append(ev_cur[i][1])
        w.append(ev_cur[i][2])
    
    tar = []   
    tar_1 = []
    tar_2 = []
    tar_3 = []
    
    for i in range(iter):
        tar.append(np.dot((I - np.dot(K[i], H)), F))
        tar_1.append(tar[i][0][0])
        tar_2.append(tar[i][1][1])
        tar_3.append(tar[i][2][2])

    plot([tar_1, tar_2, tar_3], "iter, n", "tar", "Tar matrix")
        
    return v, f, w, tar

v, f, w, tar = filter()

plot([Wdrn_noize, w], "iter, n", "Wdrn, rad/s", "Wdrn")
plot([dVe_noize, v], "iter, n", "DdVe, m/s", "dVe")
plot([Fn_noize, f], "iter, n", "Fn rad", "Fn")


In [214]:
a = []

def add():
    if True:
        print("clear a: {}".format(a))
        for i in range(4):
            a.append(i)
    
a = [1]
add()
print(a)

clear a: [1]
[1, 0, 1, 2, 3]
